# Common以下のコード補足

In [1]:
import numpy as np

***
# functions

## softmax関数
xが1次元，2次元の場合に分けて，オーバーフロー対策のために各データから最大値を引く

In [2]:
x = np.random.rand(3, 10)
print(x)
print(x.max(axis=1, keepdims=True))
print(x - x.max(axis=1, keepdims=True))

[[0.16769663 0.55759977 0.82949467 0.31966961 0.27558827 0.84324521
  0.92357382 0.44554621 0.87398662 0.08848353]
 [0.61446797 0.40254027 0.43975429 0.85685084 0.45560755 0.99538141
  0.07822269 0.96483516 0.95110046 0.73641132]
 [0.91245238 0.56621765 0.39770264 0.21816665 0.45573394 0.87416249
  0.0928468  0.80576139 0.90517761 0.95479655]]
[[0.92357382]
 [0.99538141]
 [0.95479655]]
[[-0.75587719 -0.36597405 -0.09407915 -0.60390421 -0.64798555 -0.08032861
   0.         -0.47802761 -0.0495872  -0.83509029]
 [-0.38091344 -0.59284114 -0.55562712 -0.13853057 -0.53977386  0.
  -0.91715872 -0.03054625 -0.04428094 -0.25897009]
 [-0.04234417 -0.38857889 -0.55709391 -0.7366299  -0.49906261 -0.08063406
  -0.86194974 -0.14903515 -0.04961894  0.        ]]


## cross_entropy_error
- 1データ分
$$
L = - \Sigma_k t_k \log y_k
$$

- バッチ
$$
L = - \frac{1}{N} \Sigma_n \Sigma_k t_{nk} \log y_{nk}
$$

- targetラベルがone-hot-vectorであれば，正解ラベルに対応するlog yを計算するだけ．

In [3]:
# 教師データのone-hot-vectorを正解ラベルのインデックスに変換
t = np.array([[0, 0, 1, 0],
              [1, 0, 0, 0],
              [0, 0, 0, 1]])  # target
y = np.random.rand(*t.shape)  # 入力
print(f'y_org: \n {y}')
print(f't_org: \n {t}')

batch_size = y.shape[0]  # バッチ

t = t.argmax(axis=1)  # targetをone-hot-vectorからインデックスに変換

out = -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
print(f'out: \n {out}')
print(f't_argmax: \n {t}')

y_org: 
 [[0.03351101 0.96149008 0.98055917 0.70036071]
 [0.95048979 0.11619344 0.00207229 0.43631005]
 [0.85492355 0.16384944 0.4572387  0.06011335]]
t_org: 
 [[0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]]
out: 
 0.9606438436660357
t_argmax: 
 [2 0 3]


***
# layers

## SigmoidWithLoss
$$
y = \frac{1}{1 + \exp(-x)}
\\ \\
L = - \{ t \log y + (1 - t) \log(1 - y) \}
$$
- CBOWモデルの高速化(多値分類を二値分類に置き換える)で使用．2クラス分類．
- 答えが不正解のときt = 0, 正解のときt = 1が渡される．

In [4]:
x = np.random.rand(5).reshape(5, 1)  # 入力
y = 1 / (1 + np.exp(-x))  # Sigmoid
# 以下の配列と t = 0 or 1 のラベルをcross_entropy_errorに渡せばよい
print(y)
print(np.c_[1 - y, y])

[[0.61408511]
 [0.50938914]
 [0.57505175]
 [0.57793876]
 [0.58716104]]
[[0.38591489 0.61408511]
 [0.49061086 0.50938914]
 [0.42494825 0.57505175]
 [0.42206124 0.57793876]
 [0.41283896 0.58716104]]


## Embedding
- CBOWモデルの高速化．入力層->中間層の全結合層MatMulレイヤの代わりに使う．
- MuｔMulでやったことは単語IDのone-hot-vectorと重みW_inの積和で，これはW_inから単語IDに対応するインデックスの行を抜き出すことに等しい

In [5]:
# forwardメソッド
W = np.random.rand(5, 3)  # 重み
contexts = np.array([[0, 2],
                     [1, 3],
                     [2, 4],
                     [3, 0],
                     [4, 2]])  # window_size=2, n=6のコンテキスト

print(f'W: \n {W}')

# コンテキストの数と同じだけEmbeddingレイヤを使ってコンテキストの列ごとに処理する
for i in range(contexts.shape[1]) :
    idx = contexts[:, i]
    out = W[idx]
    print(f'out_{i}: \n {out}')

W: 
 [[0.00348523 0.28101211 0.72665254]
 [0.58410279 0.69065457 0.47838342]
 [0.70680839 0.09207975 0.69834118]
 [0.67793552 0.28203948 0.15893437]
 [0.97809316 0.54394914 0.92806266]]
out_0: 
 [[0.00348523 0.28101211 0.72665254]
 [0.58410279 0.69065457 0.47838342]
 [0.70680839 0.09207975 0.69834118]
 [0.67793552 0.28203948 0.15893437]
 [0.97809316 0.54394914 0.92806266]]
out_1: 
 [[0.70680839 0.09207975 0.69834118]
 [0.67793552 0.28203948 0.15893437]
 [0.97809316 0.54394914 0.92806266]
 [0.00348523 0.28101211 0.72665254]
 [0.70680839 0.09207975 0.69834118]]


In [6]:
# backwardメソッド
grad = np.ones_like(W)  # 勾配
dW = grad
dW[...] = 0  # 勾配を初期化

dout = np.arange(W.size).reshape(W.shape)
print(f'dW_init: \n {dW}')
print(f'dout: \n {dout}')
print(f'idx: \n {contexts[:, 1]}')

np.add.at(dW, contexts[:, 1], dout)  # dWの指定した行にdoutの各行を足していく
print(f'dW: \n {dW}')

dW_init: 
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
dout: 
 [[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]]
idx: 
 [2 3 4 0 2]
dW: 
 [[ 9. 10. 11.]
 [ 0.  0.  0.]
 [12. 14. 16.]
 [ 3.  4.  5.]
 [ 6.  7.  8.]]


***
# trainer
## Trainer

In [7]:
# データのシャッフル
np.random.permutation(10)

array([6, 2, 4, 1, 8, 9, 5, 0, 3, 7])

In [8]:
x = np.random.randn(1000)
data_size = len(x)
batch_size = 32
max_iters = data_size // batch_size  # 31
max_epoch = 10
eval_interval = 20

In [9]:
# fitメソッド
for epoch in range(max_epoch):
    print(f'--------------epoch: {epoch}--------------')
    print('ここでデータをシャッフル')
    
    for iters in range(max_iters):
        print(f'--------iters: {iters}--------')
        print('ミニバッチを取得')
        print('順伝播で損失計算')
        print('逆伝播で勾配計算')
        print('max_gradを指定していれば勾配クリッピング')
        print('パラメータ更新 (optimizer.update)')
        print('total_lossに損失の値を足し上げる')
        print('loss_countに損失関数の計算回数をカウント')
        
        if (eval_interval is not None) and (iters % eval_interval) == 0:
            print('------eval------')
            print('損失の平均を計算，出力，リスト追加')
            print('total_loss, loss_countを初期化')

--------------epoch: 0--------------
ここでデータをシャッフル
--------iters: 0--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
------eval------
損失の平均を計算，出力，リスト追加
total_loss, loss_countを初期化
--------iters: 1--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 2--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 3--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 4--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 5--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
lo

--------iters: 18--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 19--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 20--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
------eval------
損失の平均を計算，出力，リスト追加
total_loss, loss_countを初期化
--------iters: 21--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 22--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 23--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 24---

順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 27--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 28--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 29--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 30--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------------epoch: 5--------------
ここでデータをシャッフル
--------iters: 0--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
------eval------
損失の平均を計算，出力，リスト追加
total_loss, loss_countを初期化
------

total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 17--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 18--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 19--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 20--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
------eval------
損失の平均を計算，出力，リスト追加
total_loss, loss_countを初期化
--------iters: 21--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 22--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げ

--------iters: 2--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 3--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 4--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 5--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 6--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 7--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (optimizer.update)
total_lossに損失の値を足し上げる
loss_countに損失関数の計算回数をカウント
--------iters: 8--------
ミニバッチを取得
順伝播で損失計算
逆伝播で勾配計算
max_gradを指定していれば勾配クリッピング
パラメータ更新 (op

In [10]:
# remove_duplicate関数
param0 = np.random.rand(3, 2)
param1 = np.random.rand(3, 2)
param2 = param0

grad0 = np.random.rand(*param0.shape)
grad1 = np.random.rand(*param1.shape)
grad2 = np.random.rand(*param2.shape)

params = [param0, param1, param2]
grads = [grad0, grad1, grad2]

print('params:')
for param in params:
    print(param)

print('')
print('grads: ')
for grad in grads:
    print(grad)

params:
[[0.44038933 0.04508724]
 [0.09261861 0.58946652]
 [0.9951653  0.17089016]]
[[0.4014445  0.82928939]
 [0.45000482 0.68067263]
 [0.84174255 0.25071469]]
[[0.44038933 0.04508724]
 [0.09261861 0.58946652]
 [0.9951653  0.17089016]]

grads: 
[[0.22577055 0.91087209]
 [0.06609719 0.35327774]
 [0.07954762 0.72539335]]
[[0.96266734 0.30767721]
 [0.93406636 0.35178235]
 [0.01985629 0.03899133]]
[[0.40932646 0.89885388]
 [0.22023787 0.73504306]
 [0.38759445 0.14625189]]


In [11]:
def remove_duplicate(params, grads):
    params, grads = params[:], grads[:]  # copy list

    while True:
        find_flg = False
        L = len(params)

        # 二つのループで全ての組み合わせをチェック
        for i in range(0, L - 1):
            for j in range(i + 1, L):
                # 重みを共有する場合
                if params[i] is params[j]:
                    grads[i] += grads[j]  # 勾配の加算
                    find_flg = True
                    params.pop(j)  # 共有する重みを削除
                    grads.pop(j)  # 共有する重みの勾配を削除
                # 転置行列として重みを共有する場合（weight tying）
                elif params[i].ndim == 2 and params[j].ndim == 2 and \
                     params[i].T.shape == params[j].shape and np.all(params[i].T == params[j]):
                    grads[i] += grads[j].T
                    find_flg = True
                    params.pop(j)
                    grads.pop(j)

                if find_flg: break
            if find_flg: break

        if not find_flg: break

    return params, grads

params_removed, grads_removed = remove_duplicate(params, grads)

In [12]:
print('params_removed: ')
for param in params_removed:
    print(param)

print('')
print('grads_removed: ')
for grad in grads_removed:
    print(grad)

params_removed: 
[[0.44038933 0.04508724]
 [0.09261861 0.58946652]
 [0.9951653  0.17089016]]
[[0.4014445  0.82928939]
 [0.45000482 0.68067263]
 [0.84174255 0.25071469]]

grads_removed: 
[[0.63509701 1.80972597]
 [0.28633506 1.08832081]
 [0.46714207 0.87164525]]
[[0.96266734 0.30767721]
 [0.93406636 0.35178235]
 [0.01985629 0.03899133]]


***
# time_layers.py
- 5章RNN以降に使うレイヤ

In [39]:
batch_size = 10  # バッチ数
vocab_size = 20  # 語彙数
wordvec_size = 5  # Embeddingで単語を分散表現にしたときの次元数．TimeRNNへの入力の次元数になる．
hidden_size = 3  # 状態ベクトルの次元数
N, V, D, H = batch_size, vocab_size, wordvec_size, hidden_size

***
## RNN
- RNNの基本的なクラス
- T個に展開することでTimeRNNレイヤを実装する

In [70]:
Wx = (np.random.randn(D, H) / np.sqrt(D)).astype('f')  # xを変換する重み
Wh = (np.random.randn(H, H) / np.sqrt(H)).astype('f')  # h_prevを変換する重み
b = np.zeros(H).astype('f')  # バイアス

In [71]:
# RNNレイヤの重み，バイアスの形状
print(f'Wh shape: {Wh.shape}')
print(f'Wx shape: {Wx.shape}')
print(f'b shape : {b.shape}')

Wh shape: (3, 3)
Wx shape: (5, 3)
b shape : (3,)


### 初期化 init

In [72]:
params = [Wx, Wh, b]  # 重みとバイアス
grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]  # 各パラメータの勾配
cache = None  # forwardの値を保持してbackwardで使うため

### forward

In [73]:
h_prev = np.random.rand(N, H)
x = np.random.rand(N, D)
print(f'h_prev shape: {h_prev.shape}')
print(f'x shape: {x.shape}')

h_prev shape: (10, 3)
x shape: (10, 5)


In [74]:
print(f'np.dot(h_prev, Wh) shape: \n {np.dot(h_prev, Wh).shape}')
print(f'np.dot(x, Wx) shape: \n {np.dot(x, Wx).shape}')

# forwardの計算
t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
h_next = np.tanh(t)
print(f'h_next shape: \n {h_next.shape}')

np.dot(h_prev, Wh) shape: 
 (10, 3)
np.dot(x, Wx) shape: 
 (10, 3)
h_next shape: 
 (10, 3)


In [75]:
cache = (x, h_prev, h_next)  # backwardで使う

### backward

In [76]:
print('========= before backward =========')
print('grads:')
print(grads)
print()

print('========= backward calculation =========')

dh_next = np.random.rand(*h_next.shape)
print(f'dh_next shape: {dh_next.shape}')

dt = dh_next * (1 - h_next ** 2)
print(f'dt shape: {dt.shape}')

db = np.sum(dt, axis=0)
dWh = np.dot(h_prev.T, dt)
dh_prev = np.dot(dt, Wh.T)
dWx = np.dot(x.T, dt)
dx = np.dot(dt, Wx.T)
print(f'db shape: {db.shape}')
print(f'dWh shape: {dWh.shape}')
print(f'dh_prev shape: {dh_prev.shape}')
print(f'dWx: {dWx.shape}')
print(f'dx: {dx.shape}')

grads[0][...] = dWx
grads[1][...] = dWh
grads[2][...] = db
print()

print('========= after backward =========')
print('grads:')
print(grads)

========= before backward =========
grads:
[array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32), array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32), array([0., 0., 0.], dtype=float32)]

========= backward calculation =========
dh_next shape: (10, 3)
dt shape: (10, 3)
db shape: (3,)
dWh shape: (3, 3)
dh_prev shape: (10, 3)
dWx: (5, 3)
dx: (10, 5)

========= after backward =========
grads:
[array([[0.5872085 , 1.0778614 , 1.5425929 ],
       [0.6233045 , 0.8048265 , 1.7906647 ],
       [0.72236395, 1.0888395 , 1.5214138 ],
       [1.0063967 , 1.1973398 , 2.353353  ],
       [0.60559374, 0.77701324, 1.7114606 ]], dtype=float32), array([[0.47196218, 0.98501956, 1.5830102 ],
       [0.5135059 , 0.60015786, 1.4544628 ],
       [0.6917176 , 0.8913182 , 1.75842   ]], dtype=float32), array([1.2954705, 1.6643497, 3.2652311], dtype=float32)]


## TimeRNNレイヤ
### 初期化 init

Wx shape: (5, 3)
Wh shape: (3, 3)
b shape : (3,)


In [34]:
params = [Wx, Wh, b]  # 重みとバイアス
grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]  # 各パラメータの勾配
layers = None  # RNNレイヤをT個まとめて管理する

h, dh = None, None  # 状態ベクトル，状態ベクトルの勾配
stateful = False  # Trueのときは状態ベクトルを引き継ぐ